# Step 1 (Enviroment Setup)


---


## Importing the Datasets
wikitext (*full* ) - 859955 docs

wikitext (*small* ) - 10000 docs

In [ ]:
!wget -O wikitext-filtered-full.zip "https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1"
!wget -O wikitext-filtered-10k.zip "https://www.dropbox.com/scl/fi/ek174r3sg7qjx0aa9atop/wikitext-filtered-10k.zip?rlkey=zy6jqxv6qsc16lr9qm3ki9uhf&dl=1"

--2025-10-18 00:19:13--  https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7cea682ee2d518730f2b216b00.dl-eu.dropboxusercontent.com/cd/0/inline/CzZkFOHnnADPhEXtKAtVphS5JS5H2q1ZUbBHVUI9n6aJI6ozm07RWKGvUUxhnb9CMpRnAbkSqhAoMf8OxXEJOgoGRGSu7rea5L6I9k0fQVoqHAJl1j0AODjDOUcFUAdjX42VbIe9XBZ4tnNdUlfiWTie/file?dl=1# [following]
--2025-10-18 00:19:15--  https://uc7cea682ee2d518730f2b216b00.dl-eu.dropboxusercontent.com/cd/0/inline/CzZkFOHnnADPhEXtKAtVphS5JS5H2q1ZUbBHVUI9n6aJI6ozm07RWKGvUUxhnb9CMpRnAbkSqhAoMf8OxXEJOgoGRGSu7rea5L6I9k0fQVoqHAJl1j0AODjDOUcFUAdjX42VbIe9XBZ4tnNdUlfiWTie/file?dl=1
Resolving uc7cea682ee2d518730f2b216b00.dl-eu.dropboxusercontent.com (uc7cea682ee2d518730f2b216b00.dl-eu.dropbo

In [ ]:
!unzip wikitext-filtered-full.zip
!unzip wikitext-filtered-10k.zip

Archive:  wikitext-filtered-full.zip
replace wikitext-filtered-full/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  wikitext-filtered-10k.zip
replace wikitext-filtered-10k/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# datasets package provides dataset tools from hugginface
!pip install datasets
import datasets

In [ ]:
from datasets import load_dataset, Dataset

def load_dataset():
  wikitext_small = "wikitext-filtered-10k"
  wikitext_large = "wikitext-filtered-full"

  dataset_small = Dataset.load_from_disk(wikitext_small)
  dataset_large = Dataset.load_from_disk(wikitext_large)
  print("wikitext_small: {} docs, wikitext_large: {} docs".format(len(dataset_small), len(dataset_large)))
  return dataset_small, dataset_large

wikitext_small, wikitext_large = load_dataset()

wikitext_small: 10000 docs, wikitext_large: 859955 docs


## Understanding the Dataset
Summary statistics

In [ ]:
wt = wikitext_small
#wt = wikitext_large

print('# TYPE OF THE DATASET:', '\n', type(wt))
print(wt, '\n')
print('# ENTRIES LOOK LIKE:')
print(wt.features, '\n', wt[0], '\n', wt[1], '\n')

print('# DATASET STATISTICS:')
print('No. of docs:', len(wt))
lengths = [len(doc['text'].split()) for doc in wt]
print('Mean doc length:', sum(lengths)/len(lengths), 'words')

# TYPE OF THE DATASET: 
 <class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['text'],
    num_rows: 10000
}) 

# ENTRIES LOOK LIKE:
{'text': Value('string')} 
 {'text': 'Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " .'} 
 {'text': "The game began development in 2010 , carrying over a large portion of the work done on Valkyria Ch

# Step 2 (Train Baselines)

---
Installing dependancies
- gensim - word2vec models
- nltk (natural language tool kit) - stopwords removal

In [ ]:
!pip install gensim nltk
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_dataset(dataset):
    text_col = 'text' if 'text' in dataset.column_names else dataset.column_names[0]
    tokenized = []

    for i in range(len(dataset)):
        text = dataset[i][text_col]
        if not isinstance(text, str):
            continue
        tokens = [t.lower() for t in text.split() if t.isalpha() and t.lower() not in stop_words]
        if tokens:
            tokenized.append(tokens)

    return tokenized

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
tokens_small = preprocess_dataset(wikitext_small)
tokens_large = preprocess_dataset(wikitext_large)

In [ ]:
def train_word2vec(tokens, model_name, vector_size=50, window=5, min_count=5, epochs=5):
    print(f"Training {model_name} ...")
    model = Word2Vec(
        sentences=tokens,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        epochs=epochs
    )
    model.save(f"{model_name}.model")
    model.wv.save(f"{model_name}.kv")
    print(f"{model_name} saved.")
    return model

In [ ]:
model_small = train_word2vec(tokens_small, "word2vec_small")
model_large = train_word2vec(tokens_large, "word2vec_large")

Training word2vec_small ...
word2vec_small saved.
Training word2vec_large ...
word2vec_large saved.


In [ ]:
# writing model to disk
model_small.save("word2vec_small.model")
model_large.save("word2vec_large.model")

In [ ]:
model_small = Word2Vec.load("word2vec_small.model")
model_large = Word2Vec.load("word2vec_large.model")

In [ ]:
vocab_size = len(model_small.wv)
print(f"Vocab size (learned by model):", vocab_size, '\n')

example_tokens = ['plane', 'car', 'planet', 'nurse', 'city', 'country']
for token in example_tokens:
    if token in model_small.wv:
        print(f"Top-10 similar to '{token}':", model_small.wv.most_similar(token, topn=10))
    else:
        print(f"'{token}' not in vocabulary.")

Vocab size (learned by model): 13838 

Top-10 similar to 'plane': [('channel', 0.9976177215576172), ('probe', 0.997555136680603), ('threatened', 0.9975177049636841), ('briefly', 0.9974639415740967), ('recovery', 0.9973675608634949), ('completion', 0.9973640441894531), ('disaster', 0.997336208820343), ('immediate', 0.9971866011619568), ('repairs', 0.9970893859863281), ('arrive', 0.9970740675926208)]
Top-10 similar to 'car': [('bring', 0.9986448287963867), ('demon', 0.9985685348510742), ('becomes', 0.9983323812484741), ('possibility', 0.9982467889785767), ('view', 0.9980642795562744), ('reality', 0.9980185627937317), ('leaves', 0.9980082511901855), ('direct', 0.997999906539917), ('fully', 0.9979443550109863), ('owen', 0.9979330897331238)]
Top-10 similar to 'planet': [('slightly', 0.9979651570320129), ('indicated', 0.9977326393127441), ('eye', 0.9974536299705505), ('classification', 0.9972626566886902), ('capable', 0.997245728969574), ('engines', 0.9970810413360596), ('offers', 0.99705696

# Task 3

In [ ]:
import numpy as np
import math

In [ ]:
def mag(v):
  s = sum((e*e) for e in v)
  s = math.sqrt(s)
  return s

def cosineSimilarity(v1, v2):
  dotProd = np.dot(v1, v2)
  cos = dotProd/(mag(v1)*mag(v2))
  return cos

In [ ]:
planeIndex = model_small.wv.key_to_index['plane']
carIndex = model_small.wv.key_to_index['car']
planetIndex = model_small.wv.key_to_index['planet']
sunIndex = model_small.wv.key_to_index['sun']
passengerIndex = model_small.wv.key_to_index['passenger']

In [ ]:
v1 = model_large.wv['planet']
v2 = model_large.wv['sun']

cosineSimilarity(v1, v2)

0.6323668823823705

# Step 4

In [ ]:
!wget -O wordsim353.zip "www.gabrilovich.com/resources/data/wordsim353/wordsim353.zip"
!unzip wordsim353.zip -d wordsim353

--2025-10-18 00:23:24--  http://www.gabrilovich.com/resources/data/wordsim353/wordsim353.zip
Resolving www.gabrilovich.com (www.gabrilovich.com)... 173.236.137.139
Connecting to www.gabrilovich.com (www.gabrilovich.com)|173.236.137.139|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gabrilovich.com/resources/data/wordsim353/wordsim353.zip [following]
--2025-10-18 00:23:24--  https://gabrilovich.com/resources/data/wordsim353/wordsim353.zip
Resolving gabrilovich.com (gabrilovich.com)... 173.236.137.139
Connecting to gabrilovich.com (gabrilovich.com)|173.236.137.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23257 (23K) [application/zip]
Saving to: ‘wordsim353.zip’

wordsim353.zip      100%[===================>]  22.71K  --.-KB/s    in 0.03s   

2025-10-18 00:23:24 (650 KB/s) - ‘wordsim353.zip’ saved [23257/23257]

Archive:  wordsim353.zip
replace wordsim353/combined.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:

In [ ]:
import pandas as pd
from scipy.stats import spearmanr

In [ ]:

df = pd.read_csv("wordsim353/combined.csv")
print("Loaded: wordsim353/combined.csv | shape:", df.shape)
print(df.head())

Loaded: wordsim353/combined.csv | shape: (353, 3)
     Word 1    Word 2  Human (mean)
0      love       sex          6.77
1     tiger       cat          7.35
2     tiger     tiger         10.00
3      book     paper          7.46
4  computer  keyboard          7.62


In [ ]:
# compare vocabularies
wordsim_vocab = set(df["Word 1"].str.lower()).union(set(df["Word 2"].str.lower()))

smallModel_vocab = set(model_small.wv.key_to_index)
largeModel_vocab = set(model_large.wv.key_to_index)

sharedVocab = wordsim_vocab.intersection(largeModel_vocab)
OOV = wordsim_vocab.difference(largeModel_vocab)

print(OOV) # should be empty

In [90]:
def sim_or_nan_strict(model, a, b):
    if a not in model.wv.key_to_index or b not in model.wv.key_to_index:
        return np.nan
    v1 = model.wv[a]
    v2 = model.wv[b]
    return float(cosineSimilarity(v1, v2))


In [91]:
def ws_spearman(model, df):
    sims = df.apply(lambda r: sim_or_nan_strict(model, r['Word 1'], r['Word 2']), axis=1)
    gold = df['Human (mean)']
    valid = sims.notna()


    if not valid.any():
        return np.nan, np.nan, 0, len(df), len(df)

    rho, p = spearmanr(sims[valid], gold[valid])

    used = int(valid.sum())
    total = int(len(df))
    n_oov = total - used
    return float(rho), float(p), used, total, n_oov


# alternative simpler implamentation
def applySpearman_to_WordSim(model, df):
  df = df.copy()
  df["Word 1"] = df["Word 1"].str.lower()
  df["Word 2"] = df["Word 2"].str.lower()

  similarities = df.apply(
      lambda r: model.wv.similarity(r["Word 1"], r["Word 2"]),
      axis = 1
  )

  human_standard = df["Human (mean)"].astype(float)
  rho, p_value = spearmanr(similarities, human_standard)

  return float(rho), float(p_value)

In [93]:
for label, m in [("wikitext_small", model_small), ("wikitext_large", model_large)]:
    rho, p, used, total, n_oov = ws_spearman(m, df)
    print(f"[{label}] Spearman ρ = {rho:.4f}")

[wikitext_small] Spearman ρ = 0.0779
[wikitext_large] Spearman ρ = 0.6342
